<a href="https://colab.research.google.com/github/NoufMQ/notebooks/blob/main/single_metahurestics_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/cs_668')

In [ ]:
!pwd

/content/drive/MyDrive/cs_668


Dataets:
  * From LR : iris , from sklearn.datasets
  * From LR :  wine,   from sklearn.datasets
  * From LR: Concrete Compressive Strength
 https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength , uploaded in drive

  * From LR : Breast Cancer Wisconsin : https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data/data , uploaded in drive

  * covertype : http://archive.ics.uci.edu/dataset/31/covertype , uploaded in drive

In [ ]:
import numpy as np
from sklearn.datasets import load_iris,load_wine
import random
from collections import deque
import time
import pandas as pd
from datetime import datetime
import math
from sklearn.metrics import silhouette_score


random.seed(1234)

def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

def calculate_sse(clusters, centroids):
    sse = 0
    for i, cluster in enumerate(clusters):
        centroid = centroids[i]
        for point in cluster:
            sse += euclidean_distance(point, centroid) ** 2
    return sse


'''def calculate_sse(clusters, centroids):
    sse = 0
    for i, cluster in enumerate(clusters):
        centroid = centroids[i]
        for point in cluster:
            sse += euclidean_distance(point, centroid) ** 2
        # Penalize for overlap
        for j, other_cluster in enumerate(clusters):
            if i != j:
                for point in other_cluster:
                    sse += euclidean_distance(point, centroid) ** 2
    return sse'''

def calculate_sse_single_cluster(clusters, centroid):
    sse = 0
    for point in cluster:
            sse += euclidean_distance(point, centroid) ** 2
    return sse

def initialize_clusters(data, k):
    n = len(data)
    indices = np.random.choice(n, k, replace=False)
    centroids = data[indices]
    clusters = [[] for _ in range(k)]
    for point in data:
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        nearest_centroid = np.argmin(distances)
        clusters[nearest_centroid].append(point)
    return clusters, centroids


def is_feasible_move(data_point, current_cluster, new_cluster,k):
    return len(new_cluster) < k and data_point not in new_cluster

def perturb(clusters):
    data_point1_index = np.random.randint(len(clusters[0]))
    data_point2_index = np.random.randint(len(clusters[1]))

    data_point1 = clusters[0][data_point1_index]
    data_point2 = clusters[1][data_point2_index]

    new_clusters = clusters.copy()
    new_clusters[0][data_point1_index] = data_point2
    new_clusters[1][data_point2_index] = data_point1

    return new_clusters

**Methods**

In [ ]:
#for ILS
def local_search(clusters, centroids,k):
    sse = calculate_sse(clusters, centroids)
    improved = True

    while improved:
        improved = False
        for i, cluster in enumerate(clusters):
            for j in range(len(cluster)):
                data_point = cluster[j]
                for new_cluster_index in range(k):
                    if new_cluster_index != i and is_feasible_move(data_point, i, clusters[new_cluster_index],k):
                        new_clusters = clusters.copy()
                        new_centroids = centroids.copy()

                        new_clusters[i].remove(data_point)

                        new_clusters[new_cluster_index].append(data_point)

                        new_centroids[i] = np.mean(new_clusters[i], axis=0) if len(new_clusters[i]) > 0 else np.zeros(len(centroids[0]))
                        new_centroids[new_cluster_index] = np.mean(new_clusters[new_cluster_index], axis=0) if len(new_clusters[new_cluster_index]) > 0 else np.zeros(len(centroids[0]))

                        new_sse = calculate_sse(new_clusters, new_centroids)

                        if new_sse < sse:
                            clusters = new_clusters.copy()
                            centroids = new_centroids.copy()
                            sse = new_sse
                            improved = True
                            break

    return clusters, centroids
#ILS
def iterated_local_search(data, k, max_iterations):

    start_time = time.time()
    silhouette_avg = -2
    n = len(data)
    dimensions = len(data[0])
    best_clusters, best_centroids = initialize_clusters(data, k)
    best_sse = calculate_sse(best_clusters, best_centroids)

    for _ in range(max_iterations):
        current_clusters, current_centroids = best_clusters.copy(), best_centroids.copy()
        current_sse = best_sse

        current_clusters = perturb(current_clusters)

        current_clusters, current_centroids = local_search(current_clusters, current_centroids,k)
        current_sse = calculate_sse(current_clusters, current_centroids)

        if current_sse < best_sse:
            best_clusters = current_clusters.copy()
            best_centroids = current_centroids.copy()
            best_sse = current_sse

    end_time = time.time()

    transformed_array = [[subarray_value, array_index] for array_index, subarray in enumerate(best_clusters) for subarray_value in subarray]
    first_column_values = [subarray_value for subarray_value, array_index in transformed_array]
    second_column_indices = [array_index for subarray, array_index in transformed_array]

    silhouette_avg = silhouette_score(first_column_values,second_column_indices)
    execution_time = end_time - start_time


    return best_sse, execution_time , silhouette_avg

#SA
def simulated_annealing_clustering(X, k,initial_temp, cooling_rate, num_iterations):
    start_time = time.time()
    silhouette_avg =-2
    n_clusters = k

    crnt_sol, crnt_centroids = initialize_clusters(X, n_clusters)
    crnt_cost = calculate_sse(crnt_sol, crnt_centroids)

    best_solution = crnt_sol
    best_centroids = crnt_centroids
    best_cost = crnt_cost

    temperature = initial_temp

    best_costs = []
    best_sse = 0

    for i in range(num_iterations):
        new_sol, new_centroids = initialize_clusters(X, n_clusters)
        new_cost = calculate_sse(new_sol, new_centroids)

        if new_cost < crnt_cost:
            crnt_sol = new_sol
            crnt_centroids = new_centroids
            crnt_cost = new_cost
        else:
            if np.random.rand() < np.exp((crnt_cost - new_cost) / temperature):
                crnt_sol = new_sol
                crnt_centroids = new_centroids
                crnt_cost = new_cost

        if crnt_cost < best_cost:
            best_solution = crnt_sol
            best_centroids = crnt_centroids
            best_cost = crnt_cost
            best_sse = crnt_cost
        temperature *= cooling_rate

        best_costs.append(best_cost)

    end_time = time.time()

    transformed_array = [[subarray_value, array_index] for array_index, subarray in enumerate(best_solution) for subarray_value in subarray]
    first_column_values = [subarray_value for subarray_value, array_index in transformed_array]
    second_column_indices = [array_index for subarray, array_index in transformed_array]

    silhouette_avg = silhouette_score(first_column_values,second_column_indices)

    execution_time = end_time - start_time

    return best_sse,execution_time, silhouette_avg

#classic LS
def hill_climbing(data, k, max_iterations):
    start_time = time.time()
    silhouette_avg = -2
    n = len(data)
    dimensions = len(data[0])

    best_clusters, best_centroids = initialize_clusters(data, k)
    best_sse = calculate_sse(best_clusters, best_centroids)

    current_clusters = best_clusters.copy()
    current_centroids = best_centroids.copy()
    current_sse = best_sse

    for _ in range(max_iterations):
        data_point_index = np.random.randint(n)
        current_cluster = next((i for i, cluster in enumerate(current_clusters) if np.array_equal(data[data_point_index], cluster)), None)

        if current_cluster is None:
            continue


        for i in range(k):
            if i != current_cluster:
                if current_cluster is not None:
                    current_clusters[current_cluster].remove(data[data_point_index])

                new_clusters = current_clusters.copy()
                new_centroids = current_centroids.copy()

                new_clusters[i].append(data[data_point_index])

                new_centroids[current_cluster] = np.mean(new_clusters[current_cluster], axis=0)
                new_centroids[i] = np.mean(new_clusters[i], axis=0)

                new_sse = calculate_sse(new_clusters, new_centroids)

                if new_sse < current_sse:
                    current_clusters = new_clusters.copy()
                    current_centroids = new_centroids.copy()
                    current_sse = new_sse

        if current_sse < best_sse:
            best_clusters = current_clusters.copy()
            best_centroids = current_centroids.copy()
            best_sse = current_sse


    end_time = time.time()


    transformed_array = [[subarray_value, array_index] for array_index, subarray in enumerate(best_clusters) for subarray_value in subarray]
    first_column_values = [subarray_value for subarray_value, array_index in transformed_array]
    second_column_indices = [array_index for subarray, array_index in transformed_array]

    silhouette_avg = silhouette_score(first_column_values,second_column_indices)

    execution_time = end_time - start_time

    return best_sse, execution_time, silhouette_avg

#TS
def tabu_search(data, k, max_iterations, tabu_size):
    start_time = time.time()
    silhouette_avg = -2
    n = len(data)
    dimensions = len(data[0])
    best_clusters, best_centroids = initialize_clusters(data, k)
    best_sse = calculate_sse(best_clusters, best_centroids)
    current_clusters = best_clusters.copy()
    current_centroids = best_centroids.copy()
    current_sse = best_sse

    tabu_list = deque(maxlen=tabu_size)

    for _ in range(max_iterations):
        data_point_index = np.random.randint(n)
        current_cluster = next((i for i, cluster in enumerate(current_clusters) if np.array_equal(data[data_point_index], cluster)), None)

        if current_cluster is None:
            continue

        possible_moves = []
        for i in range(k):
            if i != current_cluster:
                new_cluster = current_clusters[i].copy()

                if is_feasible_move(data[data_point_index], current_cluster, new_cluster,k):
                    new_clusters = current_clusters.copy()
                    new_centroids = current_centroids.copy()

                    new_clusters[i].append(data[data_point_index])

                    if current_cluster is not None:
                        new_clusters[current_cluster].remove(data[data_point_index])

                    new_centroids[current_cluster] = np.mean(new_clusters[current_cluster], axis=0)
                    new_centroids[i] = np.mean(new_clusters[i], axis=0)

                    new_sse = calculate_sse(new_clusters, new_centroids)

                    possible_moves.append((new_clusters, new_centroids, new_sse))

        best_move = None
        for move in possible_moves:
            clusters, centroids, sse = move
            if (clusters, centroids) not in tabu_list:
                if best_move is None or sse < best_sse:
                    best_move = move

        if best_move is None:
            continue

        current_clusters, current_centroids, current_sse = best_move

        tabu_list.append((current_clusters, current_centroids))

        if current_sse < best_sse:
            best_clusters = current_clusters.copy()
            best_centroids = current_centroids.copy()
            best_sse = current_sse


    end_time = time.time()

    transformed_array = [[subarray_value, array_index] for array_index, subarray in enumerate(best_clusters) for subarray_value in subarray]
    first_column_values = [subarray_value for subarray_value, array_index in transformed_array]
    second_column_indices = [array_index for subarray, array_index in transformed_array]

    silhouette_avg = silhouette_score(first_column_values,second_column_indices)

    execution_time = end_time - start_time


    return best_sse, execution_time,silhouette_avg


In [ ]:

def data_loader(choice, cases):

    data = num_cluster = 0
    match choice:
        case 1:

            data = load_iris().data
            num_cluster = len(set(load_iris().target))

        case 2:

            data = load_wine().data
            num_cluster = len(set(load_wine().target))

        case 3:

            file_ = pd.read_excel('concrete_data.xlsx')
            data = file_.iloc[:, :-1].values
            num_cluster = 7

        case 4:

            file_= pd.read_csv('breast_cancer_data.csv')
            data = file_.drop(columns=['diagnosis']).values
            num_cluster = file_['diagnosis'].nunique()

        case 5:
            file_= pd.read_csv('covertype_csv.csv')
            data = file_.iloc[:, :-1].values
            num_cluster = file_['class'].nunique()

        case _:
            print("no data has been choosen")
    return data, num_cluster


datasets_names={1: "iris_data", 2: "wine_data", 3: "concrete_data", 4: "breast_cancer_data" , 5: "covertype_data"}



In [ ]:
result=[]

In [ ]:



# Example usage
Data_choice = [1,2,3,4] #can change from 1,2,3,4,5 for datasets
max_iterations = [10, 100,1000] #can change from 10,100,1000,10000 Return to the LR to explore other number of iterations
tabu_tenure = [0.05, 0.1, 0.15 , 0.20, 0.25, 0.5]
number_of_runs = 10
best_sse = excution_time = 0

In [ ]:
for i in range(number_of_runs):
  for j in Data_choice:
      dataset_name = datasets_names.get(j)
      data , num_classes = data_loader(j, datasets_names)
      for max_i in max_iterations:

        best_sse, excution_time, silhouette_avg= hill_climbing(data, num_classes, max_i)
        result.append({
                    "dataset": dataset_name,
                    "algorithm": 'hill_climbing',
                    "best_sse": best_sse,
                    "execution_time": excution_time,
                    "max_iterations":max_i,
                    "silhouette_avg": silhouette_avg,
                    "run_no": i,
                    "parameter": 0
                })


        initial_temp = 100 #this need to be tuned
        cooling_rate = 0.95
        best_sse, excution_time, silhouette_avg= simulated_annealing_clustering(data,num_classes, initial_temp, cooling_rate, max_i)
        result.append({
                    "dataset": dataset_name,
                    "algorithm": 'simulated_annealing_clustering',
                    "best_sse": best_sse,
                    "execution_time": excution_time,
                    "max_iterations":max_i,
                    "silhouette_avg": silhouette_avg,
                    "run_no": i,
                    "parameter": [initial_temp,cooling_rate]
                })

        for t in tabu_tenure:
          best_sse, excution_time, silhouette_avg = tabu_search(data, num_classes, max_i, tabu_size=math.ceil((len(data) * t)))
          result.append({
                      "dataset": dataset_name,
                      "algorithm": 'tabu_search',
                      "best_sse": best_sse,
                      "execution_time": excution_time,
                      "silhouette_avg": silhouette_avg,
                      "max_iterations":max_i,
                      "run_no": i,
                      "parameter": [t, math.ceil((len(data) * t))]
                  })
        #print("Done with tabu_search for: ",dataset_name, " For run: ", i)


        best_sse, excution_time, silhouette_avg = iterated_local_search(data, num_classes, max_i)
        result.append({
                    "dataset": dataset_name,
                    "algorithm": 'iterated_local_search',
                    "best_sse": best_sse,
                    "execution_time": excution_time,
                    "silhouette_avg": silhouette_avg,
                    "max_iterations":max_i,
                    "run_no": i,
                    "parameter":0

                })
      print("Done with Dataset", dataset_name)
  print("Done with run no:", i)


Done with Dataset iris_data
Done with Dataset wine_data
Done with Dataset concrete_data


In [ ]:
result

In [ ]:

current_datetime = datetime.now()
filename = current_datetime.strftime("%Y-%m-%d_%H-%M-%S") + ".xlsx"
final_result = pd.DataFrame(result)
final_result.to_excel('result_'+filename)